## Market1501 / cuhk03 

In [ ]:
%matplotlib inline
import json
from pprint import pprint
Settings = json.load(open('settings.txt'))
pprint(Settings)
print('')
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '../')
from reid.stacknet import get_model
from reid.data import DataSampler

from keras.callbacks import ModelCheckpoint, TerminateOnNaN
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
from os.path import join, isfile, isdir, exists, splitext

root = Settings['data_root']

# target_w = 221
# target_h = 221
target_w = 221
target_h = 221


sampler = DataSampler(root, target_w, target_h)

npos = 3
nneg = 3
x, y = sampler.get_test_batch(npos, nneg)
#n = npos + nneg
n = len(x)
print(x.shape)

fig = plt.figure(figsize=(6, 20))
for pid in range(n):
    print('is same:', y[pid])
    im = x[pid]
    ax = fig.add_subplot(n, 2, pid*2+1); ax.axis('off')
    ax.imshow(im[:,:,0:3])
    ax = fig.add_subplot(n, 2, pid*2+2); ax.axis('off')
    ax.imshow(im[:,:,3:6])
plt.show()

## UMPM dataset

In [ ]:
from time import time
from reid.data import UMPMSampler

root = Settings['data_root']
print(root)

user = Settings['UMPM']['username']
pwd = Settings['UMPM']['password']
umpm_sampler = UMPMSampler(root, ['p2_chair_2'], user, pwd, target_w, target_h)


_start = time()
x, y = umpm_sampler.get_train()
_end = time()
print('elapsed', _end - _start)

print('same', y[0])

print(x.shape)

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(121)
ax.imshow(x[0, :, :, 0:3].astype('uint8'))
ax = fig.add_subplot(122)
ax.imshow(x[0, :, :, 3:6].astype('uint8'))
plt.show()

## MOT16 dataset

In [ ]:
from reid.data import MOT16Sampler

mot_sampler = MOT16Sampler(root, target_w, target_h)


_start = time()
x, y = mot_sampler.sample()
_end = time()
print('elapsed', _end - _start)

print('same', y[0])

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(121)
ax.imshow(x[0,:,:,0:3].astype('uint8'))
ax = fig.add_subplot(122)
ax.imshow(x[0,:,:,3:6].astype('uint8'))
plt.show()

In [ ]:
from reid.data import Data
import numpy as np
root = Settings['data_root']
user = Settings['UMPM']['username']
pwd = Settings['UMPM']['password']


sampler = Data(root, target_w, target_h, user, pwd)
print('------------')


x, y = sampler.train()

fig = plt.figure(figsize=(16,20))
for idx, im in enumerate(x):
    ax = fig.add_subplot(8, 4, idx+1); ax.axis('off')
    Im = np.concatenate([im[:,:,0:3], im[:,:,3:6]], axis=1).astype('uint8')
    ax.imshow(Im)
    ax.set_title(str(y[idx]))
    
plt.show()

In [ ]:
x, y = sampler.train()

fig = plt.figure(figsize=(16,20))
for idx, im in enumerate(x):
    ax = fig.add_subplot(8, 4, idx+1); ax.axis('off')
    Im = np.concatenate([im[:,:,0:3], im[:,:,3:6]], axis=1).astype('uint8')
    ax.imshow(Im)
    ax.set_title(str(y[idx]))
    
plt.show()

In [ ]:
from keras.applications.densenet import preprocess_input

def generate_training():
    global sampler
    while True:
        X, Y = sampler.train(add_noise=True)
        X_a = preprocess_input(X[:,:,:,0:3])
        X_b = preprocess_input(X[:,:,:,3:6])
        yield ([X_a, X_b], Y[:, 0])
        
def generate_test():
    global sampler
    while True:
        X, Y = sampler.test()
        X_a = preprocess_input(X[:,:,:,0:3])
        X_b = preprocess_input(X[:,:,:,3:6])
        yield ([X_a, X_b], Y[:, 0])

        
# ------------------------
gen = generate_training()

(Xa, Xb), Y = next(gen)

i = 0
print("y:", Y[i])

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(121); ax.axis("off")
ax.imshow(Xa[i])
ax = fig.add_subplot(122); ax.axis("off")
ax.imshow(Xb[i])

plt.show()

## Training

In [ ]:
from keras.applications.densenet import DenseNet121, preprocess_input
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, Flatten, concatenate
from keras.optimizers import RMSprop
from keras import backend as K
from keras.models import load_model

filepath = join(root, 'model_heavy.h5')
print('model:', filepath)

if isfile(filepath):
    model = load_model(filepath)
else:
    seq = Sequential()
    seq.add(DenseNet121(weights='imagenet', input_shape=(221, 221, 3),
        include_top=False, pooling='avg'))
    seq.add(Flatten())

    input_a = Input(shape=(221, 221, 3))
    input_b = Input(shape=(221, 221, 3))

    out_a = seq(input_a)
    out_b = seq(input_b)

    concatenated = concatenate([out_a, out_b])
    hidden_drp1 = Dropout(0.5)(concatenated)
    hidden = Dense(32, activation='relu')(hidden_drp1)
    hidden_drp2 = Dropout(0.5)(hidden)
    out = Dense(1, activation='sigmoid')(hidden_drp2)

    model = Model([input_a, input_b], out)

In [ ]:
from keras.optimizers import SGD, RMSprop, Nadam, Adam, Adadelta

#optimizer = Nadam()
#optimizer = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
optimizer = Nadam(lr=0.00001)
loss = 'binary_crossentropy'
metrics = ['binary_accuracy', 'acc']

model.compile(loss=loss,
              optimizer=optimizer,
              metrics=metrics)

model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, TerminateOnNaN

checkpoint = ModelCheckpoint(filepath, 
                             monitor='loss', 
                             verbose=1, 
                             save_best_only=True, mode='min')
callbacks_list = [checkpoint, TerminateOnNaN()]

history = model.fit_generator(generate_training(), 
                             validation_data=generate_test(),
                             validation_steps=5,
                             steps_per_epoch=100,
                             epochs=2000,
                             callbacks=callbacks_list)

acc = history.history['val_binary_accuracy']
tacc = history.history['binary_accuracy']

plt.plot(range(len(acc)), acc)
plt.plot(range(len(tacc)), tacc)
plt.show()

In [ ]:
# model = load_model(filepath)
# (Xa, Xb), Y = next(gen)

# Y_pred = model.predict([Xa, Xb])
# for a, b in zip(Y, Y_pred):
#     print((a, np.squeeze(b)))